In [1]:
from urllib import request
import lxml
from lxml import html
import Utils as utils
import re
from data_amount import *
import data_amount as data_amount

In [2]:
def stop_tests():
    """
    This function will test the stopping function. 
    """
    assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i/data'), True)
    assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i'), False)
    print("All Tests Passed!")
stop_tests()

<ipython-input-2-9b4c3fab3d58>:5: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i/data'), True)
<ipython-input-2-9b4c3fab3d58>:6: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(stop('https://data.cityofberkeley.info/311/311-Cases/k489-uv4i'), False)


NameError: name 'stop' is not defined

In [ ]:
def prune(url, browse_boolean):
    """
    This function will return true when the graph should continue searching
    and false when the graph should stop. 
    """
    url = url.lower().strip()
#     if 'data' not in url:
#         return False 
    if 'linkedin'in url:
        return False
    elif 'twitter' in url:
        return False
    elif 'socrata' in url:
        return False
    elif 'login' in url:
        return False
    elif 'mail' in url:
        return False
    else:
        return ((browse_boolean and (('browse' not in url) or ('page' in url))) or not browse_boolean)

In [ ]:
prune('https://data.sfgov.org/City-Management-and-Ethics/Gifts-of-Travel-Contributors/hpj3-vnwh', False)

In [3]:
def get_all_dataset_pages(url, add_to_url, max_ds_num):
    """
    This function will get all urls ending in '/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=##'
    """
    range_of_ds = range(1,(max_ds_num + 20)//10)
    ds_pages = []
    for i in range_of_ds:
        ds_pages += [(url + add_to_url + str(i))]
    return ds_pages
    

In [4]:
get_all_dataset_pages('https://data.sfgov.org/', 'browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=', 299)

['https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=1',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=2',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=3',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=4',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=5',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=6',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=7',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=8',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=9',
 'https://data.sfgov.org/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=10',
 'https://data.sfgov.org/brow

In [8]:
def search(url, add_to_url = '/browse?limitTo=datasets&utf8%2Fbrowse%3FlimitTo=datasets&utf8=&page=', browse_boolean = False, data_set_list = []):
    """
    This function will search through the 
    web graph in order to find the datasets
    """
    ds_len = data_amount.main(url)
    orig = url
    url = url + add_to_url
    url_list = []
    dataset_pages = get_all_dataset_pages(url, add_to_url, ds_len)
    pattern = re.compile(r'/([a-z]|[0-9]){4}-([a-z]|[0-9]){4}')
    
    for url in dataset_pages:
        try:
            http_response = request.urlopen(url)
            open_response = http_response.read().decode('utf-8')
            raw_html = lxml.html.fromstring(open_response)
        except:
            continue
        for link in raw_html.xpath('//a/@href'):
            if '#' in link:
                continue
            if ('limit' in link and add_to_url == ''):
                continue
            if link[-1:] == '#':
                continue
            if link in url:
                continue
            if "http" not in link:
                link = url + link
            else:
                if pattern.match(link[-10:]) is not None and 'socrata' not in link.lower():
                    print("Made it: {}/api/views/{}/rows.json?accessType=DOWNLOAD".format(orig, link[-9:]))
                    if link in data_set_list:
                        continue
                    else:
                        data_set_list += [link]
                    continue
    return data_set_list

In [9]:
ds = search('https://data.sfgov.org')
ds

Made it: https://data.sfgov.org/api/views/yitu-d5am/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/tmnf-yvry/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/g8m3-pdis/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/vw6y-z8j6/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/w4sk-nq57/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/6d9h-4u5v/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/rqzj-sfat/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/jjew-r69b/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/j2j3-acqj/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/v3c6-n4qs/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/nuek-vuh3/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/4tts-fyix/rows.json?accessType=DOWNLOAD
Made it: https:/

Made it: https://data.sfgov.org/api/views/9dsr-3f97/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/fwjv-32uk/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/6cqg-dxku/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/w3e8-bxrm/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/2wsq-7wmv/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/4phr-3hrm/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/wygs-cc76/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/4zfx-f2ts/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/65ik-7wqd/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/chfu-j7tc/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/ra2x-jzmk/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/apz9-dh7k/rows.json?accessType=DOWNLOAD
Made it: https:/

Made it: https://data.sfgov.org/api/views/mudq-s8bt/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/ez9i-q28j/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/mqks-ty4r/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/vcwn-f2xk/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/tv94-tvke/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/ni6u-knhb/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/e3kj-6yzw/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/yre9-mzkb/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/mpcm-79w2/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/htai-xw5t/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/pigs-fac7/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/sftu-nd43/rows.json?accessType=DOWNLOAD
Made it: https:/

Made it: https://data.sfgov.org/api/views/uvqv-zi4r/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/a2hm-yzs6/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/6h3c-wyfe/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/ct8k-kggq/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/8zki-wqj3/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/3g5j-aamw/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/fdm7-jqqf/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/qvpz-vw28/rows.json?accessType=DOWNLOAD
Made it: https://data.sfgov.org/api/views/vsrb-4rhy/rows.json?accessType=DOWNLOAD


['https://data.sfgov.org/Culture-and-Recreation/Film-Locations-in-San-Francisco/yitu-d5am',
 'https://data.sfgov.org/Public-Safety/-Change-Notice-Police-Department-Incidents/tmnf-yvry',
 'https://data.sfgov.org/Economy-and-Community/Registered-Business-Locations-San-Francisco/g8m3-pdis',
 'https://data.sfgov.org/City-Infrastructure/311-Cases/vw6y-z8j6',
 'https://data.sfgov.org/Health-and-Social-Services/HSA-90-day-emergency-shelter-waitlist/w4sk-nq57',
 'https://data.sfgov.org/Geographic-Locations-and-Boundaries/Street-Names/6d9h-4u5v',
 'https://data.sfgov.org/Economy-and-Community/Mobile-Food-Facility-Permit/rqzj-sfat',
 'https://data.sfgov.org/d/jjew-r69b',
 'https://data.sfgov.org/Energy-and-Environment/Existing-Commercial-Buildings-Energy-Performance-O/j2j3-acqj',
 'https://data.sfgov.org/Energy-and-Environment/Existing-Commercial-Buildings-Energy-Performance-O/v3c6-n4qs',
 'https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3',
 'https://data.sfgov.or

In [7]:
len(ds)

311

In [10]:
prog = re.compile(r'([a-z]|[0-9]){4}-([a-z]|[0-9]){4}')
prog.match(url[-9:]) is not None

True

In [25]:
pattern = re.compile(r'/([a-z]|[0-9]){4}-([a-z]|[0-9]){4}')


In [26]:
link = 'https://data.sfgov.org/City-Management-and-Ethics/Campaign-Finance-Summary-Of-Third-Party-Disclosure/7i7v-m2wn'
pattern.match('/7i7v-m2wn') is not None

True